In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/medical-insurance-dataset/Train_Data.csv
/kaggle/input/medical-insurance-dataset/Test_Data.csv


In [2]:
df = pd.read_csv('../input/medical-insurance-dataset/Train_Data.csv')
df

,age,sex,bmi,smoker,region,children,charges
0,21.000000,male,25.745000,no,northeast,2,3279.868550
1,36.976978,female,25.744165,yes,southeast,3,21454.494239
2,18.000000,male,30.030000,no,southeast,1,1720.353700
3,37.000000,male,30.676891,no,northeast,3,6801.437542
4,58.000000,male,32.010000,no,southeast,1,11946.625900
...,...,...,...,...,...,...,...
3625,48.820767,female,41.426984,no,northwest,4,10987.324964
3626,38.661977,female,26.202557,no,southeast,2,11735.844352
3627,56.000000,male,40.300000,no,southwest,0,10602.385000
3628,48.061207,female,34.930624,no,southeast,1,8976.140452


### We're trying to predict 'charges' based off the other features

In [3]:
df.isna().sum()

age         0
sex         0
bmi         0
smoker      0
region      0
children    0
charges     0
dtype: int64

# Pre-processing

In [4]:
from sklearn.preprocessing import OneHotEncoder

In [5]:
#Wrapper for gender
sex_wrapper = {'male':0, 'female':1}
df.sex = df.sex.replace(sex_wrapper)

#Wrapper for smoker
df.smoker.value_counts()
smoker_wrapper = {'no':0, 'yes':1}
df.smoker = df.smoker.replace(smoker_wrapper)

#Dealing with region - OHE
df.region.value_counts()
ohe = OneHotEncoder()
feat_array = ohe.fit_transform(df[['region']]).toarray()
df_ohe = pd.DataFrame(feat_array, columns=ohe.categories_)
df = pd.concat([df, df_ohe], axis = 1)
df = df.drop('region', axis=1)


# Splitting and training model

In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

In [7]:
y = df.charges
X = df.drop('charges', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .20, random_state=1)

scaler = StandardScaler()

s_X_train = scaler.fit_transform(X_train)
s_X_test = scaler.transform(X_test)

#Linear Regression

LR = LinearRegression()
LR.fit(s_X_train, y_train)
y_pred = LR.predict(s_X_test)

In [8]:
import math
from sklearn.metrics import mean_squared_error as mse, mean_absolute_error as mae, r2_score as r2

In [9]:
#XGBoost Regressor
xgbr = XGBRegressor(verbosity=0)
xgbr.fit(X_train, y_train)

y_pred_XGBR = xgbr.predict(X_test)

In [10]:
def get_metrics(preds, actual, model_type):
    r2s = r2(preds, actual)
    maes = mae(preds, actual)
    mses = mse(preds, actual)
    rmses = math.sqrt(mses)
#     return rmse, mse, mae, r2
    df = pd.DataFrame([r2s, maes, mses, rmses, model_type]).transpose()
    df.columns = ['r2 coeff.', 'MAE', 'MSE', 'RMSE', 'Type of model']
    return df

In [11]:
df_metrics = get_metrics(y_pred, y_test, 'Linear Regression')
df_metrics_X = get_metrics(y_pred_XGBR, y_test, 'XGBR')

df_metrics = pd.concat([df_metrics, df_metrics_X], axis=0)
df_metrics

,r2 coeff.,MAE,MSE,RMSE,Type of model
0,0.624259,3918.300193,34824927.807786,5901.264933,Linear Regression
0,0.87101,1733.224338,14806208.474594,3847.883636,XGBR


Let's improve on this a bit
# Back to the drawing-board
### Let's perform some feature engineering and outlier analysis to try and fix up our model

### 1. Outlier Analysis & EDA

In [12]:
# Work here

### 2. Feature Selection Techniques

In [13]:
# Work here

### 3. Implementing different regression models

In [14]:
# OLS Regression, Lasso Regression, Ridge Regression, 

### 4. Evaluation and comparison to original Linear Regression model and XGBRegessor model

In [15]:
# Work here